In [ ]:
import os
import cv2

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
data_train = '/content/drive/My Drive/image_data/train'

data_validation = '/content/drive/My Drive/image_data/validation'

In [ ]:
training_data_generation = ImageDataGenerator(rescale = 1.0/255,shear_range=0.2,zoom_range=0.2)


#Creating generator object of training images
training_data_generator = training_data_generation.flow_from_directory(data_train, batch_size=32, shuffle=True, class_mode='categorical', target_size=(256, 256))

In [ ]:
validation_data_generation = ImageDataGenerator(rescale = 1.0/255,shear_range=0.2,zoom_range=0.2)


#Creating generator object of training images
validation_data_generator = validation_data_generation.flow_from_directory(data_validation, batch_size=32, shuffle=True, class_mode='categorical', target_size=(256, 256))

In [ ]:
labels=['chicken','duck']

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16,decode_predictions

In [ ]:
#We are using a VGG-16 model with "imagenet" weights
model = VGG16(pooling='avg', weights='imagenet', include_top=False, input_shape=(256,256,3))

In [ ]:
#Non-trained layers
for layers in model.layers:
    layers.trainable=False
    

last_layer = model.output



vgg_new = Flatten()(last_layer)


vgg_new = Dense(128, activation = 'softmax')(vgg_new)


vgg_new = Dense(2, activation = 'softmax')(vgg_new)


vgg16_final = Model(model.input, vgg_new)


#Compiling the model
vgg16_final.compile(loss = 'categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

     

In [ ]:
vgg16_final.summary()

In [ ]:
vgg16_final.fit(training_data_generator,steps_per_epoch=len(training_data_generator),epochs=3,validation_data=validation_data_generator,validation_steps=len(validation_data_generator))

In [ ]:
vgg16_final.save('vgg_classifier.h5')

In [ ]:
from keras.models import load_model


vgg_m=load_model('vgg_classifier.h5')

Testing

In [ ]:
test ='/content/drive/My Drive/image_data/test/'

In [ ]:
check=dict()
check["image"]=[]
check["label"]=[]
for i in ["chicken","duck"]:
    for name in os.listdir(os.path.join(test,i)):
        check["image"].append(name)
        check["label"].append(i)

In [ ]:
import pandas as pd

test_data=pd.DataFrame(check)

In [ ]:
from keras.preprocessing import image

for i in range(len(test_data)): 
    image=os.path.join(test,test_data.loc[i,"label"],test_data.loc[i,"image"])
    img_pred_1=tf.keras.utils.load_img(image,target_size=(256,256))
    img_pred_1=tf.keras.utils.img_to_array(img_pred_1)
    img_pred_1=np.expand_dims(img_pred_1, axis=0)
    result= vgg16_final_model.predict(img_pred_1)
    test_data.loc[i,"test"]=np.argmax(result)

In [ ]:
for i in range(len(test_data)):
    if test_data.loc[i,"label"]=="chicken":
        test_data.loc[i,"true"]=0
    else:
        test_data.loc[i,"true"]=1

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_data["true"], test_data["pred"], target_names=["chicken","duck"]))